In [1]:
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import csv
from datetime import datetime
import numpy as np

In [2]:
# Create a dictionary that maps the run# to host_density { run#: host_density }

# paramfile = "/media/hill/DATA-LINUX/abm-data/host-density/testparams1hab7"
paramfile = "/media/hill/DATA-LINUX/abm-data/host-density-olderruns/new-model/aggregate-runs/params_05habitat"
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").replace('\n','').split(',')

        # attempt to add the density value to run, if it doesn't exist, create it
        try:
            paramd[int(result[0])].append((float(result[4])))
        except KeyError:
            paramd[int(result[0])] = (float(result[4]))  # { run#: host_density }

for k, v in paramd.items():
    print(k,v)

1 0.1
2 0.1
3 0.1
4 0.1
5 0.1
6 0.1
7 0.1
8 0.1
9 0.1
10 0.1
11 0.4
12 0.4
13 0.4
14 0.4
15 0.4
16 0.4
17 0.4
18 0.4
19 0.4
20 0.4
21 0.7
22 0.7
23 0.7
24 0.7
25 0.7
26 0.7
27 0.7
28 0.7
29 0.7
30 0.7
31 1.0
32 1.0
33 1.0
34 1.0
35 1.0
36 1.0
37 1.0
38 1.0
39 1.0
40 1.0
41 3.0
42 3.0
43 3.0
44 3.0
45 3.0
46 3.0
47 3.0
48 3.0
49 3.0
50 3.0


In [3]:
# NEW CODE - dont build so many data structures
# step 2: read csv, read df

colnames = ['run', 'tick', 'lifestate', 'total_ixodes']
# csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf3-badlines"
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density-olderruns/new-model/aggregate-runs/density-new.2020.Jul.11_hab05"
before = datetime.now()
# df = pd.read_csv(csv_file, names=colnames, error_bad_lines=False, dtype={'run':np.int32, 'tick':np.float, 'lifestate':str, 'total_ixodes':str})
df = pd.read_csv(csv_file,skiprows=8, names=colnames, error_bad_lines=False)
after = datetime.now()
print("Run time: ", after-before)
# Reading csv takes ~15-30s

Run time:  0:00:06.839521


In [4]:
# step 2.b filter df
# TODO Drop ?

# print("Dropping columns...")
# before = datetime.now()
# df.drop(['lifestate', 'tick'], axis=1, inplace=True)
# after = datetime.now()
# print("Dropping runtime: ", after-before)

print("Filtering database...")
before = datetime.now()
print("Max before filtering...", df['tick'].max())
df = df[df['tick'] < 451]
after = datetime.now()
print("Filtering runtime: ", after-before)
print("Max after filtering...", df['tick'].max())

df.head(10)


Filtering database...
Max before filtering... 1083644425.0
Filtering runtime:  0:00:01.052085
Max after filtering... 450.0


,run,tick,lifestate,total_ixodes
0,1,1.0,egg,Child 7 of Tick 7
1,1,1.0,egg,Child 95 of Tick 7
2,1,1.0,egg,Child 23 of Tick 7
3,1,1.0,egg,Child 82 of Tick 7
4,1,1.0,egg,Child 70 of Tick 7
5,1,1.0,egg,Child 29 of Tick 7
6,1,1.0,egg,Child 50 of Tick 7
7,1,1.0,egg,Child 41 of Tick 7
8,1,1.0,egg,Child 45 of Tick 7
9,1,1.0,egg,Child 3 of Tick 7


In [ ]:
# Doesn't seem to be much performance from dropping benefit in this case. Test on larger files
# df.drop(['tick', 'lifestate'], axis=1, inplace=True)
# df.head(10)

In [5]:
print("Groupby and agg...")
before = datetime.now()
n_ticks_df = df.groupby(['run'], as_index=False)
# n_ticks = n_ticks.agg(total_ixodes=pd.NamedAgg(column='name', aggfunc='nunique')) # namedAgg example
n_ticks_df = n_ticks_df.agg({'total_ixodes': 'nunique'})
after = datetime.now()
print("Groupby runtime: ", after-before)
# takes ~14s

Groupby and agg...
Groupby runtime:  0:00:05.630644


In [6]:
n_ticks_df.head()

,run,total_ixodes
0,1,5709
1,2,3010
2,3,18010
3,4,6010
4,5,24010


In [7]:
# step 3
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    n_ticks_df.loc[n_ticks_df['run']==key, 'host_density' ] = float(value) #FIXME? check theres no off by 1
n_ticks_df.head()


,run,total_ixodes,host_density
0,1,5709,0.1
1,2,3010,0.1
2,3,18010,0.1
3,4,6010,0.1
4,5,24010,0.1


In [11]:
# Step 4 make final df
# Works, BUT it's giving me sample and I want population (even with np.std ?)

final_df = n_ticks_df.groupby('host_density')['total_ixodes'].agg({'mean', 'std'})
final_df.head()


,std,mean
host_density,,
0.1,8054.140694,9029.9
0.4,8769.819823,10171.2
0.7,5440.695390,7152.9
1.0,5116.821658,8363.9
3.0,6236.417380,6565.2


In [ ]:
# Step 5 - write to csv
# could add some kind of 'first file' flag to change mode between write and append
final_df.to_csv('/media/hill/DATA-LINUX/abm-data/host-density-olderruns/new-model/aggregate-runs/dfagg', mode='w')

In [ ]:
# example of how to plot
plt.errorbar(final_df.index, final_df['mean'], yerr=2*final_df['std'])
plt.plot(marker='o')
plt.show()

In [ ]:
# # OLD step 4 make the final df
# n_ticks = n_ticks.groupby('host_density', )
# n_ticks = n_ticks.agg({'total_ixodes': 'mean'})
# n_ticks.head()

In [ ]:
# mean = n_ticks.groupby('host_density')['total_ixodes'].mean()
# std = n_ticks.groupby('host_density')['total_ixodes'].std()
# mean

In [ ]:
# How to get rid of the bad lines ------------
print(df['tick'].max())
before = datetime.now()
new_df = df[df['tick'] < 451 ]
after = datetime.now()
print("Run time: ", after-before)
print(new_df['tick'].max())
print(new_df.head())

In [ ]:
df.drop(['tick','lifestate'], axis=1, inplace=True)
df.head()
# How to get rid of the bad lines ------------

In [ ]:
# -- OLD way
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
colnames = ['run', 'tick', 'lifestate', 'name']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf1hab"
df = pd.read_csv(csv_file, names=colnames, header=None, error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())


In [ ]:
# Creating a data frame from the cumulative_ixodes dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixodes'])
df_final['host_density'] = 0
df_final.dtypes

In [ ]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(paramd)):
        # print(type(value))
        df_final.loc[df_final['run']==key, 'host_density'] = value

df_final.dtypes

In [ ]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_final.groupby('host_density'):
    tmp_df = density[1] # density is a tuple, so density[1] is the df we want
    # print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# in test file - should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


In [ ]:
# Create a dataframe from the agg_ixodes_dict that we can then plot
df_agg_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_agg_final.head()

In [ ]:
matplotlib.use('Qt5Agg')
plt.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.show(block=True)

In [ ]:
with open(paramfile, 'r') as file:
    reader = csv.reader(file)
    param_str = {rows[0]: "Lifestage: {}\nStarting Ixodes: {}\nHabitat Suitability: {}".format(
        rows[1], rows[2], rows[3]) for rows in reader}
    print(param_str)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Test Agg Host Density")
ax.set_ylabel("Total Ixodes")
ax.set_xlabel("Host Density")
props = dict(facecolor='wheat', alpha=0.5)
ax.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.figtext(0.5,0.5, 'Lifestage: adult\nStarting Ixodes: 10\nHabitat Suitability: 0.05', bbox=props)
plt.show(block=True)

In [ ]:
df2 = df_agg_final.copy()
ax=df2.plot()
df_agg_final.plot(ax=ax)
plt.show(block=True)



In [ ]:
df2 = df_agg_final.copy()
ax=df2.plot()
df_agg_final.plot(ax=ax)
plt.show(block=True)



In [ ]:
df.drop(['tick','lifestate'], axis=1, inplace=True)
df.head()
# How to get rid of the bad lines ------------

In [ ]:
# -- OLD way
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
colnames = ['run', 'tick', 'lifestate', 'name']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf1hab"
df = pd.read_csv(csv_file, names=colnames, header=None, error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())


In [ ]:
# Creating a data frame from the cumulative_ixodes dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixodes'])
df_final['host_density'] = 0
df_final.dtypes

In [ ]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(paramd)):
        # print(type(value))
        df_final.loc[df_final['run']==key, 'host_density'] = value

df_final.dtypes

In [ ]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_final.groupby('host_density'):
    tmp_df = density[1] # density is a tuple, so density[1] is the df we want
    # print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# in test file - should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


In [ ]:
# Create a dataframe from the agg_ixodes_dict that we can then plot
df_agg_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_agg_final.head()

In [ ]:
matplotlib.use('Qt5Agg')
plt.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.show(block=True)

In [ ]:
with open(paramfile, 'r') as file:
    reader = csv.reader(file)
    param_str = {rows[0]: "Lifestage: {}\nStarting Ixodes: {}\nHabitat Suitability: {}".format(
        rows[1], rows[2], rows[3]) for rows in reader}
    print(param_str)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Test Agg Host Density")
ax.set_ylabel("Total Ixodes")
ax.set_xlabel("Host Density")
props = dict(facecolor='wheat', alpha=0.5)
ax.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.figtext(0.5,0.5, 'Lifestage: adult\nStarting Ixodes: 10\nHabitat Suitability: 0.05', bbox=props)
plt.show(block=True)

In [ ]:
df2 = df_agg_final.copy()
ax=df2.plot()
df_agg_final.plot(ax=ax)
plt.show(block=True)

